## Watch the [YouTube video](https://www.youtube.com/watch?v=mM_dC1HVAQ4) first for more context

In [ ]:
import numpy as np
import random
import os
import csv
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import math
import urllib.request
import json

## Collecting the data

### Set the boundaries

Since we are just collecting data for the USA, we want to set the latitude and longitude boundaries to be restricted to the USA. These values have been multiplied by 10,000,000.

In [ ]:
lat_min = 247433195
lat_max = 493457868
lon_min = -1247844079
lon_max = -669513812

### Street view API parameters

This project uses the [Google Street View API](https://developers.google.com/maps/documentation/streetview/overview) to collect street view images. The parameters are set according to [this doc](https://developers.google.com/maps/documentation/streetview/request-streetview). Note that you will need to obtain your own API key by following [this guide](https://developers.google.com/maps/documentation/streetview/cloud-setup).

**Scraping data using the Google Street View API is against Google's terms of service. Use at your own risk. Google may be able to tell that you are scraping data and give your account a warning.**

In [ ]:
key = '&key=YOUR-API-KEY'
size = '&size=640x640'
radius = '&radius=1000'
source = '&source=outdoor'

### Downloading the images

The following code will download 1000 streetview images from random points in within the USA. As it collects the images it will also record the image name, latitude, and longitude to a CSV file, which will be needed later. It first calls the [Street View Metadata API](https://developers.google.com/maps/documentation/streetview/metadata) to ensure that the image is valid. If the image is valid, then it will call the actual Street View API and save the image to the disk.

In [ ]:
PICTURES_TO_COLLECT = 1000
IMAGE_DIRECTORY = 'images/USA/'
ANNOTATION_CSV = 'annot.csv'

with open(ANNOTATION_CSV, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    while i < PICTURES_TO_COLLECT:
        lat = random.randrange(lat_min, lat_max) / 10000000
        lng = random.randrange(lng_min, lng_max) / 10000000

        location = '&location=' + str(lat) + ',' + str(lng)

        parameters = size + location + radius + source + key
        metadata_url = 'https://maps.googleapis.com/maps/api/streetview/metadata?' + parameters

        response = urllib.request.urlopen(metadata_url, timeout=30)
        data = response.read()
        encoding = response.info().get_content_charset('utf-8')
        metadata = json.loads(data.decode(encoding))

        if metadata['status'] == 'OK':
            streetview_url = 'https://maps.googleapis.com/maps/api/streetview?' + parameters
            name = 'USA_' + str(i) + '.jpg'
            urllib.request.urlretrieve(streetview_url, IMAGE_DIRECTORY + name)
            writer.writerow([name, str(metadata['location']['lat']), str(metadata['location']['lng'])])
            i+=1

## Define boxes

The USA will be divided into multiple boxes which will be used to guess where a given streetview is located. The bounding box which was used to collect the images does not perfectly match the USA. Because of this, we define the boxes which are within the bounding box, but not in the USA. We will exclude images that were collected from these boxes. 

In [ ]:
lat_min = 24.7433195
lat_max = 49.3457868
lon_min = -124.7844079
lon_max = -66.9513812

# Divide the country into 50 total boxes. 10 boxes wide and 5 boxes tall.
x_boxes = 10
y_boxes = 5

# These boxes are outside of the USA
outside = [[0,0], [1,0], [2,0], [3,0], [5,0], [6,0], [8,0], [9,0],
           [0,1], [8,1], [9,1], [9,2], [7,4], [8,4]]

box_width = (lon_max - lon_min) / x_boxes
box_height = (lat_max - lat_min) / y_boxes

## Load annotations

Using the CSV containing the annotations for the images, the annotations are loaded into memory as long as the image is within the USA.

In [ ]:
im_names = os.listdir(IMAGE_DIRECTORY)
all_ims = []
with open(ANNOTATION_CSV, newline='') as datafile:
    reader = csv.reader(datafile, delimiter=',')
    for row in reader:
        if row[0] in im_names:
            lat = float(row[1])
            lon = float(row[2])
            row_n = math.floor((lat-lat_min) / box_height)
            col_n = math.floor((lon-lon_min) / box_width)
            if row_n >= y_boxes or col_n >= x_boxes:
                continue
            if [col_n, row_n] in outside:
                continue
            all_ims.append([row[0], float(row[1]), float(row[2])])

## Initialize ML models

### Pretrained model

This project uses feature extraction and transfer learning. View [this tutorial](https://developers.google.com/machine-learning/practica/image-classification) for a hands on example of how convolutional neural networks work, and how we can use feature extraction. The pretrained model we will be using for feature extraction is [MobileNetV2](https://keras.io/api/applications/mobilenet/#mobilenetv2-function). The input to this model will be streetview image.

In [ ]:
input_width = 224
input_height = 224
pre_trained_model = keras.applications.MobileNetV2(
    input_shape=(input_width, input_height, 3), 
    include_top=False,
    weights='imagenet', 
    pooling='avg')

pre_trained_model.summary()

### Classifier

We will define a simple classifier model which takes the output features of the pretrained model as inputs. This model will output a final guess for the location of the image.

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(50, activation='softmax')
])

## Preprocess images

### Load images

The following code loads all of the images into a numpy array and labels all of the images with the corresponding box they belong to.

In [ ]:
im_batch = []
labels = []

random.shuffle(all_ims)

for im_info in all_ims:
    im = Image.open(IMAGE_DIRECTORY + im_info[0]).resize((input_width, input_height))
    im_batch.append(np.array(im))
    lat = im_info[1]
    lon = im_info[2]
    row_n = math.floor((lat-lat_min) / box_height)
    col_n = math.floor((lon-lon_min) / box_width)
    label = [0 for i in range(50)]
    label[row_n * x_boxes + col_n] = 1
    labels.append(label)
            
labels = np.array(labels)
im_batch = np.array(im_batch)

### Extract features

The images are preprocessed for the MobileNetV2 model and are then inputed into the model. The output of the model is the extracted features for all of the images. These features will be used as input to the classifier model.

In [ ]:
im_batch = keras.applications.mobilenet_v2.preprocess_input(im_batch)
data = pre_trained_model.predict(im_batch, verbose=1)

## Train model

The classifier model is trained on the output of the pretrained model. The model does not need to train for many epochs before the model starts to overfit.

In [ ]:
BATCH_SIZE = 1024
EPOCHS = 20
VALIDATION_SPLIT = 0.2

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='acc')

history = model.fit(
    data,
    labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT
)

## Evaluate model

Evaluate the loss and accuracy of the model after it finishes training, and plot the data.

In [ ]:
# Retrieve a list of loss results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Get range of epochs
epochs_range = range(EPOCHS)

# Plot training and validation accuracy per epoch
plt.plot(epochs_range, acc, label='Training')
plt.plot(epochs_range, val_acc, label='Validation')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.legend()

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs_range, loss, label='Training')
plt.plot(epochs_range, val_loss, label='Validation')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.legend()

## Save model

This command saves the model on the disk and can be used again in other programs.

In [ ]:
model.save('geoguessr_model.h5')

## Use model

### Load image

Load a streetview image to test with the model. Run the image through the pretrained model and then send the features through the newly trained model.

In [ ]:
base_im = Image.open('images/test_image.jpg').resize((input_width, input_height))
im_p = np.array([np.array(base_im)])
im_p = keras.applications.mobilenet_v2.preprocess_input(im_p)
features_p = pre_trained_model.predict(im_p)
prediction = model.predict(features_p)[0]

plt.imshow(base_im)

### Make guess

The following code will plot a map of the USA and place the guess as a pink dot on the map. It shades each box on the map based on the confidence of the AI that the image belongs to that box. The darker the box, the more confident the AI is that the image belongs to that box. The final guess is made by taking a weighted average of each box. The more confident the AI is that the image belongs to a box, the closer the final guess will be to that box.

In [ ]:
model = keras.models.load_model('geoguessr_model.h5')

plt.figure(figsize=(24,12))
map = Basemap(projection='mill', llcrnrlon=lon_min, llcrnrlat=lat_min, urcrnrlon=lon_max, urcrnrlat=lat_max)
map.drawmapboundary(fill_color='aqua')
map.drawcountries()
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()

centers = [0 for _ in range(x_boxes*y_boxes)]
for i in range(x_boxes):
    for j in range(y_boxes):
        confidence = prediction[j*x_boxes+i]
        
        x_orig = box_width * i
        y_orig = box_height * j
        x_offset = box_width * (i+1)
        y_offset = box_height * (j+1)
        lons = [lon_min+x_orig, lon_min+x_offset, lon_min+x_offset, lon_min+x_orig]
        lats = [lat_min+y_orig, lat_min+y_orig, lat_min+y_offset, lat_min+y_offset]
        centers[j*x_boxes+i] = [sum(lats)/4, sum(lons)/4]

        x, y = map(lons, lats)
        xy = zip(x,y)
        poly = Polygon( list(xy), fill=True, linewidth=2, facecolor='black', alpha=confidence, edgecolor='black')
        plt.gca().add_patch(poly)
        
lat_p = 0
lon_p = 0
for i in range(x_boxes*y_boxes):
    lat_p += prediction[i] * centers[i][0]
    lon_p += prediction[i] * centers[i][1]
x, y = map(lon_p, lat_p)
map.plot(x, y, marker='o',color='m', zorder=10, markersize=10)
    
print(str(lat_p) + ', ' + str(lon_p))
    
plt.show()